In [1]:
pip install transformers datasets evaluate seqeval

Note: you may need to restart the kernel to use updated packages.


# Task

Token classification assigns a label to individual tokens in a sentence. One of the most common token classification tasks is Named Entity Recognition (NER). NER attempts to find a label for each entity in a sentence, such as a person, location, or organization.

This notebook shows how to:
1. Finetune DistilBERT on the WNUT 17 dataset to detect new entities.
2. Use the finetuned model for inference.

# Libraries

In [2]:
import torch
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import 
from transformers import AutoTokenizer, DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, pipeline

mps_device = torch.device("mps")

2024-01-29 21:23:50.170112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data

In [3]:
# Load the WNUT 17 dataset from the 🤗 Datasets library
wnut = load_dataset("wnut_17")

In [4]:
# Check out an example
wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [5]:
tokens_list = wnut["train"][0]['tokens']
ner_tags_list = wnut["train"][0]['ner_tags']
print(tuple(zip(tokens_list, ner_tags_list)))

(('@paulwalk', 0), ('It', 0), ("'s", 0), ('the', 0), ('view', 0), ('from', 0), ('where', 0), ('I', 0), ("'m", 0), ('living', 0), ('for', 0), ('two', 0), ('weeks', 0), ('.', 0), ('Empire', 7), ('State', 8), ('Building', 8), ('=', 0), ('ESB', 7), ('.', 0), ('Pretty', 0), ('bad', 0), ('storm', 0), ('here', 0), ('last', 0), ('evening', 0), ('.', 0))


In [6]:
# Each number in ner_tags represents an entity
# Convert the numbers to their label names to find out what the entities are
# NB: The letter that prefixes each ner_tag indicates the token position of the entity:
# B - indicates the beginning of an entity.
# I - token is contained inside the same entity (e.g. State token is a part of an entity like Empire State Building).
# 0 indicates the token doesn’t correspond to any entity
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

# Preprocessing

In [7]:
# Load a DistilBERT tokenizer to preprocess the tokens field
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [8]:
# From the example tokens field above, it looks like the input has already been tokenized...
# But the input actually hasn’t been tokenized yet
# We’ll need to set is_split_into_words=True to tokenize the words into subwords
example = wnut["train"][0]
example_tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
example_tokens = tokenizer.convert_ids_to_tokens(example_tokenized_input["input_ids"])
example_tokens

['[CLS]',
 '@',
 'paul',
 '##walk',
 'it',
 "'",
 's',
 'the',
 'view',
 'from',
 'where',
 'i',
 "'",
 'm',
 'living',
 'for',
 'two',
 'weeks',
 '.',
 'empire',
 'state',
 'building',
 '=',
 'es',
 '##b',
 '.',
 'pretty',
 'bad',
 'storm',
 'here',
 'last',
 'evening',
 '.',
 '[SEP]']

In [9]:
# Note that the tokenisation process above introduces special characters such as [SEP] and [CLS]
# Issue: this creates a mismatch between the input and labels. 
# Need to realign the tokens and labels

def tokenize_and_align_labels(examples):
    # Map all tokens to their corresponding word with the word_ids method
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    # Assign the label -100 to the special tokens [CLS] and [SEP]
    # This causes them to be ignored by the PyTorch loss function (CrossEntropyLoss).
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
# apply the preprocessing function over the entire dataset
# speed up the map function by setting batched=True to process multiple elements of the dataset at once
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

In [11]:
# create a batch of examples using DataCollatorWithPadding
# It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation
# Avoid pre-padding the whole dataset to the maximum length
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# Evaluation

In [12]:
# Load the load the seqeval framework to compute several scores: precision, recall, F1, and accuracy
seqeval = evaluate.load("seqeval")

In [13]:
# Get the NER labels... 
# then create a function that passes true predictions and true labels to seqeval
labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training

In [14]:
# First create id/label mappings
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [15]:
# Call pre-trained model and pass id/label mappings as parameters
model = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=len(label2id), id2label=id2label, label2id=label2id
)

model.to(mps_device)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [16]:
training_args = TrainingArguments(
    output_dir="token_classification_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
#trainer.push_to_hub()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.082492,0.509950,0.189991,0.276840,0.936429
2,No log,0.075771,0.570698,0.310473,0.402161,0.941046


/Users/nm/opt/anaconda3/envs/nlp/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=426, training_loss=0.0774017119071853, metrics={'train_runtime': 116.4029, 'train_samples_per_second': 58.315, 'train_steps_per_second': 3.66, 'total_flos': 91781128898820.0, 'train_loss': 0.0774017119071853, 'epoch': 2.0})

# Inference

In [17]:
# Text to run inference on
text = "Only in downtown Los Angeles will you find The Reserve Nightclub and a Paul Smith store in the same street."

In [18]:
# Simplest way to try out the finetuned model for inference is to use it in a pipeline()
classifier = pipeline("ner", model="token_classification_model/checkpoint-426")
classifier(text)

[{'entity': 'B-location',
  'score': 0.40168393,
  'index': 3,
  'word': 'downtown',
  'start': 8,
  'end': 16},
 {'entity': 'B-location',
  'score': 0.6298744,
  'index': 4,
  'word': 'los',
  'start': 17,
  'end': 20},
 {'entity': 'B-location',
  'score': 0.5008956,
  'index': 5,
  'word': 'angeles',
  'start': 21,
  'end': 28},
 {'entity': 'B-location',
  'score': 0.26978534,
  'index': 10,
  'word': 'reserve',
  'start': 47,
  'end': 54},
 {'entity': 'B-person',
  'score': 0.6892672,
  'index': 14,
  'word': 'paul',
  'start': 71,
  'end': 75},
 {'entity': 'I-person',
  'score': 0.62078613,
  'index': 15,
  'word': 'smith',
  'start': 76,
  'end': 81}]

# Training Pipeline 

In [19]:
# Tokenise the text and return PyTorch tensors
tokenizer = AutoTokenizer.from_pretrained("token_classification_model/checkpoint-426")
inputs = tokenizer(text, return_tensors="pt")

In [20]:
# Pass your inputs to the model and return the logits
model = AutoModelForTokenClassification.from_pretrained("token_classification_model/checkpoint-426")
with torch.no_grad():
    logits = model(**inputs).logits

In [21]:
# Get the class with the highest probability
# Use the model’s id2label mapping to convert it to a text label
predictions = torch.argmax(logits, dim=2)
predicted_token_class = [model.config.id2label[t.item()] for t in predictions[0]]
predicted_token_class

['O',
 'O',
 'O',
 'B-location',
 'B-location',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'O',
 'O',
 'O',
 'B-person',
 'I-person',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']